In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 2000)
pd.set_option('display.colheader_justify', "left")

In [2]:
# crime_path = r"C:\Users\NCC200\Desktop\TASK\crime_work\archive.zip"
# df = pd.read_csv(crime_path, low_memory=True, compression='zip')
# df.tail()

In [3]:
def load_and_optimize_csv_clean(filepath, sample_size=5000, chunksize=100_000):
    sample = pd.read_csv(filepath, nrows=sample_size, low_memory=False)

    dtypes = {}
    for col in sample.columns:
        if sample[col].dtype == "object":
            dtypes[col] = "category"
        elif pd.api.types.is_integer_dtype(sample[col]):
            dtypes[col] = "Int32"
        elif pd.api.types.is_float_dtype(sample[col]):
            dtypes[col] = "float32"
        else:
            dtypes[col] = sample[col].dtype

    chunks = []
    for chunk in pd.read_csv(filepath, dtype=dtypes, chunksize=chunksize, low_memory=False):
        chunk = chunk.dropna().drop_duplicates()
        chunks.append(chunk)

    df_clean = pd.concat(chunks, ignore_index=True)
    return df_clean


In [4]:
df = load_and_optimize_csv_clean(r"C:\Users\NCC200\Desktop\TASK\crime_work\archive.zip")

# Create a cleaned copy
#df_clean = df.dropna().drop_duplicates()

print("Original shape:", df.shape)
#print("Cleaned shape:", df_clean.shape)

Original shape: (7084435, 22)


In [5]:
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,924,9,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.669998,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,1511,15,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895081,-87.765404,"(41.895080471, -87.765400451)"
2,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,1412,14,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937405,-87.716652,"(41.937405765, -87.716649687)"
3,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,1522,15,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881905,-87.755119,"(41.881903443, -87.755121152)"
4,10224742,HY411435,09/05/2015 10:55:00 AM,082XX S LOOMIS BLVD,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,614,6,21.0,71.0,05,1168430.0,1850165.0,2015,02/10/2018 03:50:01 PM,41.744377,-87.658432,"(41.744378879, -87.658430635)"


In [6]:
df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude', 'Location'], dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7084435 entries, 0 to 7084434
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    Int32  
 1   Case Number           object 
 2   Date                  object 
 3   Block                 object 
 4   IUCR                  object 
 5   Primary Type          object 
 6   Description           object 
 7   Location Description  object 
 8   Arrest                bool   
 9   Domestic              bool   
 10  Beat                  Int32  
 11  District              Int32  
 12  Ward                  float32
 13  Community Area        float32
 14  FBI Code              object 
 15  X Coordinate          float32
 16  Y Coordinate          float32
 17  Year                  Int32  
 18  Updated On            object 
 19  Latitude              float32
 20  Longitude             float32
 21  Location              object 
dtypes: Int32(4), bool(2), float32(6), object(1

In [8]:
df.isna().sum()

ID                      0
Case Number             0
Date                    0
Block                   0
IUCR                    0
Primary Type            0
Description             0
Location Description    0
Arrest                  0
Domestic                0
Beat                    0
District                0
Ward                    0
Community Area          0
FBI Code                0
X Coordinate            0
Y Coordinate            0
Year                    0
Updated On              0
Latitude                0
Longitude               0
Location                0
dtype: int64

In [9]:
df.nunique()

ID                      7084435
Case Number             7083910
Date                    2935205
Block                     37417
IUCR                        401
Primary Type                 35
Description                 540
Location Description        215
Arrest                        2
Domestic                      2
Beat                        303
District                     24
Ward                         50
Community Area               78
FBI Code                     26
X Coordinate              75460
Y Coordinate             125705
Year                         23
Updated On                 4332
Latitude                  92686
Longitude                 38541
Location                 672492
dtype: int64

In [10]:
arrest_district = df.groupby("District")["Arrest"].value_counts(normalize=True).mul(100).unstack(fill_value=0).reset_index()
arrest_district

Arrest,District,False,True
0,1,73.019543,26.980457
1,2,75.338328,24.661672
2,3,74.608049,25.391951
3,4,77.620185,22.379815
4,5,74.833079,25.166921
5,6,74.237058,25.762942
6,7,72.020585,27.979415
7,8,77.494531,22.505469
8,9,72.625356,27.374644
9,10,68.529217,31.470783


In [11]:
arrest_district_domestic = df.groupby("District")["Domestic"].value_counts(normalize=True).mul(100).unstack(fill_value=0).reset_index()
arrest_district_domestic

Domestic,District,False,True
0,1,95.127069,4.872931
1,2,86.238855,13.761145
2,3,79.213077,20.786923
3,4,84.027284,15.972716
4,5,80.514718,19.485282
5,6,80.875958,19.124042
6,7,79.852301,20.147699
7,8,85.466762,14.533238
8,9,87.069945,12.930055
9,10,84.010890,15.989110


In [12]:
arrest_ward = df.groupby("Ward")["Arrest"].value_counts(normalize=True).mul(100).unstack(fill_value=0).reset_index()
arrest_ward

Arrest,Ward,False,True
0,1.0,80.295352,19.704648
1,2.0,70.281067,29.718933
2,3.0,67.084868,32.915132
3,4.0,78.296450,21.703550
4,5.0,78.367649,21.632351
5,6.0,75.337271,24.662729
6,7.0,76.640410,23.359590
7,8.0,79.147739,20.852261
8,9.0,75.124641,24.875359
9,10.0,76.986342,23.013658


In [13]:
arrest_ward_domestic = df.groupby("Ward")["Domestic"].value_counts(normalize=True).mul(100).unstack(fill_value=0).reset_index()
arrest_ward_domestic

Domestic,Ward,False,True
0,1.0,92.699684,7.300316
1,2.0,91.776275,8.223725
2,3.0,87.410493,12.589507
3,4.0,85.905007,14.094993
4,5.0,81.257021,18.742979
5,6.0,81.544341,18.455659
6,7.0,81.249935,18.750065
7,8.0,81.763767,18.236233
8,9.0,80.146194,19.853806
9,10.0,84.907692,15.092308


In [14]:
year_arrest = df.groupby("Year")["Arrest"].value_counts(normalize=True).mul(100).unstack(fill_value=0).reset_index()
year_arrest

Arrest,Year,False,True
0,2001,81.404321,18.595679
1,2002,72.126159,27.873841
2,2003,70.245024,29.754976
3,2004,69.121576,30.878424
4,2005,68.829968,31.170032
5,2006,69.770303,30.229697
6,2007,69.764564,30.235436
7,2008,74.240699,25.759301
8,2009,71.796480,28.203520
9,2010,72.834061,27.165939


In [15]:
year_domestic = df.groupby("Year")["Domestic"].value_counts(normalize=True).mul(100).unstack(fill_value=0).reset_index()
year_domestic

Domestic,Year,False,True
0,2001,92.952675,7.047325
1,2002,87.390805,12.609195
2,2003,88.688651,11.311349
3,2004,88.388721,11.611279
4,2005,88.800493,11.199507
5,2006,88.519166,11.480834
6,2007,88.333808,11.666192
7,2008,88.114647,11.885353
8,2009,86.427851,13.572149
9,2010,85.995322,14.004678


In [16]:
description_list = df['Description'].value_counts()
description_list

Description
SIMPLE                                                          783695
DOMESTIC BATTERY SIMPLE                                         606759
$500 AND UNDER                                                  548691
TO VEHICLE                                                      394683
TO PROPERTY                                                     372788
OVER $500                                                       368858
AUTOMOBILE                                                      270364
FORCIBLE ENTRY                                                  260212
POSS: CANNABIS 30GMS OR LESS                                    256847
FROM BUILDING                                                   251615
RETAIL THEFT                                                    202427
TELEPHONE THREAT                                                136415
TO LAND                                                         119329
UNLAWFUL ENTRY                                                  1

In [17]:
primary_type_list = df['Primary Type'].value_counts()
primary_type_list

Primary Type
THEFT                                1499197
BATTERY                              1299859
CRIMINAL DAMAGE                       811905
NARCOTICS                             669097
ASSAULT                               465810
OTHER OFFENSE                         440288
BURGLARY                              390418
MOTOR VEHICLE THEFT                   339630
DECEPTIVE PRACTICE                    302833
ROBBERY                               267994
CRIMINAL TRESPASS                     195986
WEAPONS VIOLATION                     100385
PROSTITUTION                           61348
OFFENSE INVOLVING CHILDREN             49456
PUBLIC PEACE VIOLATION                 48705
SEX OFFENSE                            26311
CRIM SEXUAL ASSAULT                    24123
INTERFERENCE WITH PUBLIC OFFICER       17821
GAMBLING                               13405
LIQUOR LAW VIOLATION                   12782
HOMICIDE                               12168
ARSON                                  118

In [23]:
location_description = df['Location Description'].value_counts()
location_description

Location Description
STREET                                                   1825181
RESIDENCE                                                1165529
APARTMENT                                                 831792
SIDEWALK                                                  687096
OTHER                                                     239180
PARKING LOT/GARAGE(NON.RESID.)                            181751
ALLEY                                                     158415
SMALL RETAIL STORE                                        136575
SCHOOL, PUBLIC, BUILDING                                  131382
RESIDENCE-GARAGE                                          122142
RESTAURANT                                                117525
VEHICLE NON-COMMERCIAL                                    116778
RESIDENCE PORCH/HALLWAY                                   112266
DEPARTMENT STORE                                           89796
GROCERY FOOD STORE                                         87418
GAS 